In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import re
import pickle
# %matplotlib inline

In [2]:
with open('../data/mini-clean.pickle', 'rb') as handle:
    df = pickle.load(handle)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1763530 entries, 0 to 1763529
Data columns (total 28 columns):
date                          object
companyId                     object
jobId                         object
country                       object
stateProvince                 object
city                          object
avgOverallRating              float64
numReviews                    float64
industry                      object
normTitle                     object
normTitleCategory             object
descriptionCharacterLength    int64
descriptionWordCount          int64
experienceRequired            float64
estimatedSalary               float64
salaryCurrency                object
jobLanguage                   object
supervisingJob                float64
licenseRequiredJob            float64
educationRequirement          object
jobAgeDays                    int64
clicks                        int64
localClicks                   float64
month                         int

In [4]:
df2 = df[['industry', 'normTitleCategory']]
df2.head(20)

,industry,normTitleCategory
0,NaN,childcare
1,NaN,sales
2,NaN,engcivil
3,NaN,hr
4,HEALTH_CARE,mednurse
5,NaN,protective
6,NaN,transport
7,NaN,engid
8,NaN,media
9,NaN,education


In [33]:
## Remove 2nd industry Testing one
text = df2.industry[35]
text
p = r'^[A-Z_]+'
m = re.search(p, text)
text[m.start(): m.end()]

'CONSULTING_AND_BUSINESS_SERVICES,HUMAN_RESOURCES_AND_STAFFING'

'CONSULTING_AND_BUSINESS_SERVICES'

In [60]:
def get_first_industry(text):
    if type(text) == str: 
        p = r'^[A-Z_]+'
        m = re.search(p, text)
        return text[m.start():m.end()]
    else:
        return np.nan

In [64]:
texts = df2.industry
len(texts.unique())
firsts = [get_first_industry(t) for t in texts]


195

AttributeError: 'list' object has no attribute 'unique'

In [84]:
df2.industry = firsts
df2.head()

/mnt/c/Users/fredcorpuz06/miniconda3/envs/kaggle/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,industry,normTitleCategory
0,NaN,childcare
1,NaN,sales
2,NaN,engcivil
3,NaN,hr
4,HEALTH_CARE,mednurse


In [85]:
df3 = pd.get_dummies(df2, columns=['normTitleCategory'])
df3.head()

,industry,normTitleCategory_accounting,normTitleCategory_admin,normTitleCategory_agriculture,normTitleCategory_arch,normTitleCategory_arts,normTitleCategory_aviation,normTitleCategory_care,normTitleCategory_childcare,normTitleCategory_construction,...,normTitleCategory_sports,normTitleCategory_tech,normTitleCategory_techhelp,normTitleCategory_techinfo,normTitleCategory_techsoftware,normTitleCategory_therapy,normTitleCategory_transport,normTitleCategory_uncategorized,normTitleCategory_veterinary,normTitleCategory_warehouse
0,NaN,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HEALTH_CARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
ind_missing = df3.industry.isnull()
df3_wo = df3[ind_missing]
print(len(df3_wo))

df3_w = df3[np.logical_not(ind_missing)]
print(len(df3_w))

1581661
181869


In [96]:
X = df3_w.iloc[:,1:].values
y = df3_w.industry.values
print(X.shape, y.shape)

(181869, 57) (181869,)


In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(121852, 57) (60017, 57) (121852,) (60017,)


In [107]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.externals import joblib

In [124]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# model = DecisionTreeClassifier()
model = RandomForestClassifier(n_estimators=100)

In [122]:
RandomForestClassifier?

In [125]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [126]:
y_pred = model.predict(X_test)

In [120]:
confusion_matrix(y_test, y_pred)

array([[   28,     0,     0,     3,     0,     6,   235,     0,     9,
            0,     0,     2,   237,    74,     0,     4,     0,     0,
            0,     0,     0,     0,    61,     0,     0],
       [    0,    11,     0,     1,     0,    22,    30,     0,     4,
            0,     0,     0,   170,    55,     1,     0,     0,     0,
            0,     0,     0,     2,    45,     0,    48],
       [    0,     0,     0,     8,     0,    18,    87,     0,     3,
            0,     0,     0,   617,   118,     0,     9,     0,     0,
            0,     0,     4,    11,   613,     0,   145],
       [    4,     0,     0,   167,     0,     1,   356,     2,    14,
            0,     0,     0,   687,   201,     0,    31,     0,     0,
            0,     0,    17,    11,  1114,     0,     2],
       [    0,     0,     0,     1,     0,     0,   107,     0,     2,
            0,     0,     0,    89,    18,     0,     0,     0,     0,
            0,     0,     0,     3,    52,     0,     1],


In [121]:
classification_report(y_test, y_pred)

/mnt/c/Users/fredcorpuz06/miniconda3/envs/kaggle/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'                                  precision    recall  f1-score   support\n\n           AEROSPACE_AND_DEFENSE       0.60      0.04      0.08       659\n      AGRICULTURE_AND_EXTRACTION       0.35      0.03      0.05       389\n                            AUTO       0.00      0.00      0.00      1633\n    BANKS_AND_FINANCIAL_SERVICES       0.46      0.06      0.11      2607\n       COMPUTERS_AND_ELECTRONICS       0.00      0.00      0.00       273\n                    CONSTRUCTION       0.28      0.12      0.16      1639\nCONSULTING_AND_BUSINESS_SERVICES       0.23      0.27      0.25      4808\n     CONSUMER_GOODS_AND_SERVICES       0.54      0.08      0.14      1545\n           EDUCATION_AND_SCHOOLS       0.64      0.35      0.45      3841\n            ENERGY_AND_UTILITIES       0.00      0.00      0.00       750\n              FOOD_AND_BEVERAGES       0.00      0.00      0.00      1611\n                      GOVERNMENT       0.84      0.01      0.02      1557\n                     H

In [127]:
f1_score(y_test, y_pred, average='weighted')

/mnt/c/Users/fredcorpuz06/miniconda3/envs/kaggle/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3415676279778798